## Cliff's re-implementation of the Deffuant Model, in Python

In [ ]:
# a very simple implementation of Relative Agreement, 
# aimed just at replicating Figure 9

import random
from math import sqrt

#range of uncertainty (U) values for moderate agents
u_min = 0.2
u_max = 2.0
#number of steps this u-range is divided into
u_steps = 19 #this was 19 in the original Fig 9

#range of global proportion of extremists (pe)
pe_min = 0.025
pe_max = 0.3
#number of steps the pe-range is divided into
pe_steps = 12 #this was 12 in the original Fig 9

#number of iid repetitions of the simulation at each (u,pe) point
sims_per_point = 50

#number of runs to dump as timeseries of opinion for each agent at each (u,pe)
n_dumps = 0

#opinion values
Max_Op = 1.0
Min_Op = -1.0

#population size
N = 200

#expected number of iterations per individual
n_iter = 250

#intensity of interactions
mu = 0.2

#extremism uncertainty
u_e = 0.1

#DC introduced this parameter:
#how close does the opinion value have to be to one of the extremes to count as extreme?
extreme_distance = 0.2

Min_mod_op=Min_Op+extreme_distance
Max_mod_op=Max_Op-extreme_distance    

#calculate y value for a given u, pe pair
#dump is a flag that determines whether to write the timeseries of opinion values to file
#dumpid is the id of the dump file

def calc_y(u, pe, dump, dumpid):

    if(dump>0):
        # write population values to file on this run
        filename='RA'+"_%2.3f"%u+"_%2.3f"%pe+"_%2d"%dumpid+'.csv'
        outfile=open(filename,'w')


    # generate initial population of individuals
    pop=[]
    for i in range(N):
        Op=Min_mod_op+(random.random()*(Max_mod_op-Min_mod_op))
        #pop structure is [opinion,certainty,start_opinion,n_iterations]
        pop.append([Op,u,"moderate",0])
        

    # use pe to determine number of extremists (and hence number of moderates)
    n_extremists=pe*N
    N_P_plus=int(0.5+(n_extremists/2.0))
    #assume symmetric plus/minus
    N_P_minus=N_P_plus
    n_moderate=N-(N_P_plus+N_P_minus)

    # create extremists: Max then Min
    for i in range(N_P_plus):
        pop[i][0]=Max_Op-(random.random()*extreme_distance)          
        pop[i][1]=u_e
        pop[i][2]="extreme"
               
    for i in range(N_P_minus):
        pop[N_P_plus+i][0]=Min_Op+(random.random()*extreme_distance)          
        pop[N_P_plus+i][1]=u_e
        pop[N_P_plus+i][2]="extreme"


    # this is the main experiment loop
    for iter in range(n_iter):

        # write the line of agent opinions for this iteration
        if(dump>0):
            outstr=""
            for i in range(N):
                if(i<(n-1)):
                    outstr+="%2.3f"%pop[i][0]+", "
                else:
                    outstr+="%2.3f"%pop[i][0]+"\n"
            outfile.write(outstr)    
            

        #now do n updates        
            
        for i in range(n):
            
            # randomly pick an individual to be influenced
            agent=random.randint(0,N-1)

            # randomly pick a different individual to be the influencer
            influencer=agent
            while(influencer==agent):
                influencer=random.randint(0,N-1)

            # allow them to interact and possibly alter opinions
            # switching to variable names used by deffuant et al
            x_j=pop[agent][0]
            u_j=pop[agent][1]
            x_i=pop[influencer][0]
            u_i=pop[influencer][1]
            h_ij=min(x_i+u_i, x_j+u_j) - max(x_i-u_i, x_j-u_j)
            if(h_ij>u_i):
                relagree=(h_ij/u_i)-1
                delta_x_j=mu*relagree*(x_i-x_j)
                delta_u_j=mu*relagree*(u_i-u_j)
                pop[agent][0]=x_j+delta_x_j
                pop[agent][1]=u_j+delta_u_j
                # print "influence! dx, du=",delta_x_j,delta_u_j
            
            # update the counts
            pop[agent][3]=pop[agent][3]+1
            pop[influencer][3]=pop[influencer][3]+1    
        

    # now do the post-experiment analysis
    # compare current opinion and initial opinion
    p_prime_plus_count=0
    p_prime_minus_count=0

    for i in range(N):

        # increment p_prime_plus_count if this is a moderate that went +ve extreme
        if((pop[i][2]=="moderate")&((Max_Op-pop[i][0])<=extreme_distance)):
            p_prime_plus_count+=1;

        # increment p_prime_minus_count if this is a moderate that went -ve extreme
        if((pop[i][2]=="moderate")&((pop[i][0]-Min_Op)<=extreme_distance)):
            p_prime_minus_count+=1
      

    p_prime_plus=p_prime_plus_count/float(n_moderate)
    p_prime_minus=p_prime_minus_count/float(n_moderate)

    y=(p_prime_plus*p_prime_plus)+(p_prime_minus*p_prime_minus)
    print "y=",y

    #write the number of interactions each agent has had
    if(dump>0):
        outstr="\n"
        for i in range(N):
            if(i<(n-1)):
                outstr+="%3d"%pop[i][3]+", "
            else:
                outstr+="%3d"%pop[i][3]+"\n"
                outfile.write(outstr)  
        
        outfile.close()
        
    return y



#calculate min, max, avg & stdev of y value for a given u, pe pair over n trials

def calc_y_stats(u,pe):
    y_max=-10.0
    y_min=10.0
    y_sum=0.0
    y_sumsq=0.0
    for sim in range(sims_per_point):
        if(sim<n_dumps):
            dump=1
        else:
            dump=0
        y=calc_y(u, pe, dump, sim)
        y_sum=y_sum+y
        y_sumsq=y_sumsq+(y*y)
        if(y>y_max):
            y_max=y
        if(y<y_min):
            y_min=y
    y_avg=y_sum/sims_per_point
    y_sd=sqrt((y_sumsq/sims_per_point)-(y_avg*y_avg))
    return (y_max, y_min, y_avg, y_sd)


#main: calc avg + sd for y at each (u,pe) point

u_delta=(u_max-u_min)/(u_steps-1)
pe_delta=(pe_max-pe_min)/(pe_steps-1)
mindata=[]
maxdata=[]
avgdata=[]
sdvdata=[]

for u_i in range(u_steps):
    u=u_min+(u_i*u_delta)
    minrow=[]
    maxrow=[]
    avgrow=[]
    sdvrow=[]
    for pe_i in range(pe_steps):
        pe=pe_min+(pe_i*pe_delta)
        y_stats=calc_y_stats(u, pe)
        print (u_i, pe_i), (u, pe), y_stats
        minrow.append(y_stats[1])
        maxrow.append(y_stats[0])
        avgrow.append(y_stats[2])
        sdvrow.append(y_stats[3])
    mindata.append(minrow)
    maxdata.append(maxrow)
    avgdata.append(avgrow)
    sdvdata.append(sdvrow)

#print it all out
outfile=open('RAout_min.csv','w')
for u_i in range(u_steps):
    outstr=""
    for pe_i in range(pe_steps):
        if(pe_i<(pe_steps-1)):
           outstr+="%2.4f"%mindata[u_i][pe_i]+", "
        else:
           outstr+="%2.4f"%mindata[u_i][pe_i]+"\n"
           
    outfile.write(outstr)    
    print outstr
outfile.close()

#print it all out
outfile=open('RAout_max.csv','w')
for u_i in range(u_steps):
    outstr=""
    for pe_i in range(pe_steps):
        if(pe_i<(pe_steps-1)):
           outstr+="%2.4f"%maxdata[u_i][pe_i]+", "
        else:
           outstr+="%2.4f"%maxdata[u_i][pe_i]+"\n"
           
    outfile.write(outstr)    
    print outstr
outfile.close()

#print it all out
outfile=open('RAout_avg.csv','w')
for u_i in range(u_steps):
    outstr=""
    for pe_i in range(pe_steps):
        if(pe_i<(pe_steps-1)):
           outstr+="%2.4f"%avgdata[u_i][pe_i]+", "
        else:
           outstr+="%2.4f"%avgdata[u_i][pe_i]+"\n"
           
    outfile.write(outstr)    
    print outstr
outfile.close()

#print it all out
outfile=open('RAout_stdev.csv','w')
for u_i in range(u_steps):
    outstr=""
    for pe_i in range(pe_steps):
        if(pe_i<(pe_steps-1)):
           outstr+="%2.4f"%sdvdata[u_i][pe_i]+", "
        else:
           outstr+="%2.4f"%sdvdata[u_i][pe_i]+"\n"
           
    outfile.write(outstr)    
    print outstr
outfile.close()